## **네이버뉴스 카카오톡 전송 프로젝트**

### **모듈 패키지 설치**

In [23]:
!pip install gensim BeautifulSoup4 lxml requests selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **네이버뉴스 url 크롤링하기**

In [24]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import json

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    # 임시 이미지
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    
     # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
                + "&sid1=" \
                + sid
    print("section url : ", sec_url)
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img
        }
        news_list3.append(news_info)
        
    return news_list3

In [25]:
# 경제', '사회', 'IT' 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    # 뉴스 결과를 담아낼 dictionary
    news_dic = dict()
    
    # sections : 경제', '사회', "IT"
    sections = ["eco", "soc", "IT"]
    # section_ids :  URL에 사용될 뉴스  각 부문 ID
    section_ids = ["101", "102", "105"]
    
    for sec, sid in zip(sections, section_ids):   
        # 뉴스의 기본 정보 가져오기
        news_info = get_top3_news_info(sec, sid)
        # print(news_info)
        for news in news_info:
            # 뉴스 url 가져오기
            news_url = news['news_url']

        news_dic[sec] = news_info
    
    return news_dic
    
# 함수 호출 - '경제', '사회', 'IT' 분야의 상위 3개 뉴스 크롤링
# news_dic 에 분야별로 3개씩 기사 넣기
news_dic = get_naver_news_top3()

print("*"*100)

# news_dic 결과 확인하기 
news_dic

section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101
section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=102
section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=105
****************************************************************************************************


{'eco': [{'title': '환율, 18원 이상 급등 1340원대 등락…中 코로나 불안, 위험회피',
   'date': '1분전',
   'news_url': 'https://n.news.naver.com/mnews/article/018/0005376226?sid=101',
   'image_url': 'https://imgnews.pstatic.net/image/origin/018/2022/11/28/5376226.jpg?type=nf106_72'},
  {'title': ' (인사) 세방',
   'date': '1분전',
   'news_url': 'https://n.news.naver.com/mnews/article/021/0002543707?sid=101',
   'image_url': 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#'},
  {'title': '정부 "가상자산 과세 2년 미뤄야"…시행 한달 앞두고 국회 넘을까',
   'date': '1분전',
   'news_url': 'https://n.news.naver.com/mnews/article/008/0004822989?sid=101',
   'image_url': 'https://imgnews.pstatic.net/image/origin/008/2022/11/28/4822989.jpg?type=nf106_72'}],
 'soc': [{'title': "'정남진' 전남 장흥, '정북진' 북한 중강진과 교류 추진",
   'date': '1분전',
   'news_url': 'https://n.news.naver.com/mnews/article/001/0013607084?sid=102',
   'image_url': 'https://imgnews.pstatic.net/image/origin/001/2022/11/28/13607084.jpg?type=nf106_72'},
  {'title': '軍

### **gensim뉴스 요약 #1**

In [26]:
# 1
from gensim.summarization.summarizer import summarize

# 섹션 지정 (경제, 사회, IT중 하나를 입력)
my_section = input('eco, soc, IT 중 하나를 입력하세요 : ')
news_list3 = news_dic[my_section]

# 뉴스 요약하기
for news_info in news_list3:
    
    # 뉴스 타이틀이 20 문자 초과일 경우 생략. 즉, 최대 20문자까지만 출력.
    try:
        snews_contents = summarize(news_info['title'], word_count=20)
    except:
        snews_contents = None

    news_info['snews_contents'] = snews_contents

# 요약한 news_list3 출력    
news_list3

eco, soc, IT 중 하나를 입력하세요 : IT


[{'title': '"불충분한 소명, 훼손된 신뢰가 뭔지 알려달라"…위메이드, 닥사에 호소',
  'date': '2분전',
  'news_url': 'https://n.news.naver.com/mnews/article/421/0006486602?sid=105',
  'image_url': 'https://imgnews.pstatic.net/image/origin/421/2022/11/28/6486602.jpg?type=nf106_72',
  'snews_contents': None},
 {'title': "화해, 연말 총결산 세일 '화해위크' 진행",
  'date': '3분전',
  'news_url': 'https://n.news.naver.com/mnews/article/092/0002274932?sid=105',
  'image_url': 'https://imgnews.pstatic.net/image/origin/092/2022/11/28/2274932.jpg?type=nf106_72',
  'snews_contents': None},
 {'title': '위메이드 “닥사 입장문 사실과 달라…소명 넘어 증명까지 해”',
  'date': '4분전',
  'news_url': 'https://n.news.naver.com/mnews/article/366/0000858260?sid=105',
  'image_url': 'https://imgnews.pstatic.net/image/origin/366/2022/11/28/858260.jpg?type=nf106_72',
  'snews_contents': None}]

### **카카오톡 토큰 발급**

In [28]:
# 카카오 개발자 사이트
print("https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api#request-code")

# rest api키
# fc5b9e33753fe1189b74fc26be6e18b1


# 인증코드 받기
# https://kauth.kakao.com/oauth/authorize?client_id=fc5b9e33753fe1189b74fc26be6e18b1&response_type=code&redirect_uri=https://localhost.com

https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api#request-code


In [29]:
import requests
import json

url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : "fc5b9e33753fe1189b74fc26be6e18b1",
    "redirect_uri" : "https://localhost.com",
    "code"         : "vgbK2OGtuEcOzRI7UscQ3TtZ1jCLZkuGBrHIrhi1GK1ybqdkUSmyEodwLVz1rlAWlX4ECAo9dVwAAAGEvORo8g"
    
}
response = requests.post(url, data=data)

tokens = response.json()

print(tokens)

{'access_token': 'ffYt91AvDvPhttDk_q-1nUfjDTMLRHVWGaaLIU7kCisM1AAAAYS85Sg8', 'token_type': 'bearer', 'refresh_token': 'aEQyaG8_XHPT-rIkW0HHp098BE6Z0EXbnF9CzwBLCisM1AAAAYS85Sg7', 'expires_in': 21599, 'scope': 'account_email profile_image talk_message profile_nickname', 'refresh_token_expires_in': 5183999}


In [30]:
with open("kakao_token.json", "w") as fp:
    json.dump(tokens, fp)

In [37]:
# 모듈 패키지 불러와서 저장하기

from google.colab import files
src = list(files.upload().values())[0]
open('file1.py','wb').write(src)
import file1

Saving kakao_utils.py to kakao_utils.py


In [38]:
# 생성한 토큰 접근하기

import json
import kakao_utils

KAKAO_TOKEN_FILENAME = "/content/kakao_token.json"
KAKAO_APP_KEY = "fc5b9e33753fe1189b74fc26be6e18b1"
kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

{'access_token': 'InRR3QqQL7gvxQoSvKS_gZEv-s43Ej8e5j8m35FsCinI2gAAAYS86Mmy', 'token_type': 'bearer', 'expires_in': 21599}


{'access_token': 'InRR3QqQL7gvxQoSvKS_gZEv-s43Ej8e5j8m35FsCinI2gAAAYS86Mmy',
 'token_type': 'bearer',
 'refresh_token': 'aEQyaG8_XHPT-rIkW0HHp098BE6Z0EXbnF9CzwBLCisM1AAAAYS85Sg7',
 'expires_in': 21599,
 'scope': 'account_email profile_image talk_message profile_nickname',
 'refresh_token_expires_in': 5183999}

### **템플릿 만들기 #2**

In [39]:
# 2 
# 사용자가 선택한 카테고리를 제목에 넣기 위한 dictionary
sections_ko = {'eco' : '경제','soc' : '사회', 'IT' : 'IT/과학'}

# 네이버 뉴스 URL
navernews_url = "https://news.naver.com/main/home.nhn"

# 추후 각 리스트에 들어갈 내용(content) 만들기
contents = []

# 리스트 템플릿 형식 만들기
template = {
    "object_type" : "list",
    "header_title" : sections_ko[my_section] + " 분야 상위 뉴스 빅3",
    "header_link" : {
        "web_url": navernews_url,
        "mobile_web_url" : navernews_url
    },
    "contents" : contents,
    "button_title" : "네이버 뉴스 바로가기"
}

## 내용 만들기
# 각 리스트에 들어갈 내용(content) 만들기
for news_info in news_list3:    
    content = {
        "title" : news_info.get('title'),
        "description" : "작성일 : " + news_info.get('date'),
        "image_url" : news_info.get('image_url'),
        "image_width" : 50, "image_height" : 50,
        "link": {
            "web_url": news_info.get('news_url'),
            "mobile_web_url": news_info.get('news_url')
        }
    }
    
    contents.append(content)
    

### **카카오 메시지 전송 코드 #3**

In [40]:
# 3
# 카카오 메시지 전송
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
if res.json().get('result_code') == 0:
    print('뉴스를 성공적으로 보냈습니다.')
else:
    print('뉴스를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

뉴스를 성공적으로 보냈습니다.


분야별 뉴스기사 보낼때 : 1.gensim뉴스요약 -> 2.템플릿 만들기 -> 3.카카오 메시지

## **번외**

In [ ]:
# 3번에 걸쳐 각 뉴스 속보 요약 결과를 전송합니다
for idx, news_info in enumerate(news_list3):
    # 텍스트 템플릿 형식 만들기
    template = {
      "object_type": "text",
      "text": '# 제목 : ' + news_info.get('title'),
      "link": {
            "web_url": news_info.get('news_url'),
            "mobile_web_url": news_info.get('news_url')
      },
      "button_title": "자세히 보기"
    }
    
    # 카카오 메시지 전송
    res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
    if res.json().get('result_code') == 0:
        print('뉴스를 성공적으로 보냈습니다.')
    else:
        print('뉴스를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

뉴스를 성공적으로 보냈습니다.
뉴스를 성공적으로 보냈습니다.
뉴스를 성공적으로 보냈습니다.


### **온라인 뉴스 데이터 추출**

In [51]:
import pandas as pd

In [52]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
DATA_PATH = '/gdrive/My Drive/Colab Notebooks/2022_DMC/dmc_idea_project/data/'

Mounted at /gdrive


In [53]:
df_news = pd.read_csv(DATA_PATH+'온라인뉴스.csv')

In [58]:
df_news.head()

,사업자등록번호,순번,데이터수집일자,언론사명,기사날짜,기사제목,기사내용,기자명,URL링크,작업구분코드,데이터생성일자,연계처리상태코드,연계처리일자
0,1240400001,1,20220408.0,NaN,NaN,NaN,NaN,NaN,NaN,I,20221114,NaN,NaN
1,1242100002,1,20220408.0,매일경제,201910.0,2020년 ICT 10대 이슈는...5G보호무역주의AI,NaN,NaN,NaN,I,20221114,NaN,NaN
2,1171400003,1,20220408.0,NaN,NaN,NaN,NaN,NaN,NaN,I,20221114,NaN,NaN
3,1212000004,1,20220408.0,이투데이,201908.0,"주간 주요 일정 거시경제금융회의 개최, 소재부품 수급 대응 지원센터 운영 현황, 밀...",NaN,NaN,NaN,I,20221114,NaN,NaN
4,1274000005,1,20220408.0,서울경제,202204.0,"단독 웹젠 노조, 게임업계 최초로 파업 결정",NaN,NaN,NaN,I,20221114,NaN,NaN


In [59]:
df_news.columns

Index(['사업자등록번호', '순번', '데이터수집일자', '언론사명', '기사날짜', '기사제목', '기사내용', '기자명',
       'URL링크', '작업구분코드', '데이터생성일자', '연계처리상태코드', '연계처리일자'],
      dtype='object')

In [70]:
df_news2 = df_news.dropna(subset=['기사내용'])
df_news2.head()

,사업자등록번호,순번,데이터수집일자,언론사명,기사날짜,기사제목,기사내용,기자명,URL링크,작업구분코드,데이터생성일자,연계처리상태코드,연계처리일자
13,3148200012,1,20210914.0,에너지경제,201807.0,"재단법인 한국연구재단, 7월 26일목 13시까지 2018년 하반기 체험형 청년인...","관심기사얘는 내 친척 아닌데추석 연휴 서먹한 친족들, 법에서는 몇촌까지가 적당할까간...",NaN,NaN,I,20221114,NaN,NaN
14,3148200012,2,20210914.0,에너지경제,201603.0,"재단법인 한국연구재단, 4월 8일금 17시까지 인턴 모집","관심기사얘는 내 친척 아닌데추석 연휴 서먹한 친족들, 법에서는 몇촌까지가 적당할까간...",NaN,NaN,I,20221114,NaN,NaN
37,1088100031,3,20210607.0,뉴시스,201801.0,인터뷰잘하는 것에 집중해야수호랑 만든 이희곤 대표,서울뉴시스평창 동계올림픽 마스코트인 수호랑을 디자인한 캐릭터 개발 전문업체 매스씨앤...,NaN,NaN,I,20221114,NaN,NaN
40,1088100031,6,20210607.0,뉴시스,201708.0,항노화의 중심 산삼골 함양 스토리텔링 밑그림 완성,함양뉴시스정경규 기자 경남 함양군은 8일 소회의실에서 스토리텔링 보고회를 열고있다...,NaN,NaN,I,20221114,NaN,NaN
41,1088100031,7,20210607.0,전자신문,201906.0,"희망일출 산행팀, 2019 릴레이 통일 희망일출 진행루게릭요양병원 건립 및...",3년 넘게 루게릭요양병원 건립 후원을 위한 등반에 나서고 있는 희망일출 산행팀대장 ...,NaN,NaN,I,20221114,NaN,NaN


In [85]:
df_news2.loc[df_news2['사업자등록번호'] == 3148200012]['기사제목'].values[0]

'재단법인 한국연구재단, 7월 26일목 13시까지 2018년 하반기 체험형 청년인...'

In [86]:
template = {
    "object_type" : "text",
    "title" : "뉴스 기사 토픽 현황",
    "text" : df_news2.loc[df_news2['사업자등록번호'] == 3148200012]['기사제목'].values[0],
    "link" : {
        "web_url": "https://news.naver.com/",
        "mobile_web_url" : "https://news.naver.com/"
    }
}

In [87]:
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
if res.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

메시지를 성공적으로 보냈습니다.
